In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

## (2)データの読み込み

In [2]:
email_data = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')


## (3)ルールによるメールの配信を行ったログを作成

In [3]:
## データの成形とrunning variable の追加
male_data = email_data[email_data.segment.isin(['Mens E-Mail', 'No E-Mail'])].copy()
male_data['treatment'] = male_data.segment.apply(lambda x: 1 if x=='Mens E-Mail' else 0)
male_data['history_log'] = np.log(male_data.history)

In [11]:
## cut off の値を指定
threshold_value = 5.5
## ルールによる介入を再現したデータを作成
## cut off より running variable が大きければ配信されたデータのみ残す
## 逆の場合には配信されなかったデータのみ残す
## running variable を 0.1 単位で区切ったグループわけの変数も追加しておく
male_data['history_log_grp'] = np.round(male_data.history_log / 0.1) * 0.1
rdd_data = male_data[
    ((male_data.history_log > threshold_value) & (male_data.segment == 'Mens E-Mail')) |
    ((male_data.history_log <= threshold_value) & (male_data.segment == 'No E-Mail'))
]

## (4)RCTデータとRDDデータの傾向の比較
### running variable とサイト来訪率のプロット（RCTデータ）

In [5]:
summarised = male_data.groupby(['history_log_grp', 'segment']).agg(visit=('visit', 'mean'), N=('visit', 'count')).reset_index()
summarised = summarised[summarised.N > 10]

In [6]:
fig = px.scatter(summarised, x='history_log_grp', y='visit', color='segment', symbol='segment', size='N'
                 , title='5.2 実験データにおける来訪率とlog(history_i)')
fig.show()

In [7]:
fig.write_html('../images/ch5_plot1.html')

FileNotFoundError: [Errno 2] No such file or directory: '../images/ch5_plot1.html'

### running variable とサイト来訪率のプロット（RDDデータ）

In [12]:
summarised = rdd_data.groupby(['history_log_grp', 'segment']).agg(visit=('visit', 'mean'), N=('visit', 'count')).reset_index()
summarised = summarised[summarised.N > 10]

In [13]:
fig = px.scatter(summarised, x='history_log_grp', y='visit', color='segment', symbol='segment', size='N'
                 , title='5.3 非実験データにおける来訪率とlog(history_i)')
fig.add_trace(go.Scatter(
    x=[threshold_value, threshold_value],
    y=[0, 0.35],
    mode="lines",
    line=dict(color='grey', dash='dot'),
    name='cut-off value'
))
fig.show()

In [ ]:
fig.write_html('../images/ch5_plot2.html')

## (5)集計による分析

In [15]:
## RCTデータでの比較
male_data[
          (male_data.history_log > 5) & (male_data.history_log < 6)
].groupby("treatment").agg(count=('visit','count'), visit_rate=('visit','mean'))

,count,visit_rate
treatment,,
0,7167,0.12125
1,7135,0.20042


In [16]:
## RDDデータでの比較
rdd_data.groupby('treatment').agg(count=('visit', 'count'), visit_rate=('visit', 'mean'))

,count,visit_rate
treatment,,
0,13926,0.090694
1,7366,0.224002


## (6)回帰分析による分析

In [17]:
## 線形回帰による分析
y = rdd_data.visit
X = rdd_data[['treatment', 'history_log']]
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
coef = results.summary().tables[1]
coef = pd.read_html(coef.as_html(), header=0, index_col=0)[0]
pd.DataFrame(coef.loc['treatment']).T

,coef,std err,t,P>|t|,[0.025,0.975]
treatment,0.1137,0.008,14.24,0.0,0.098,0.129


## (7)分析に使うデータの幅と分析結果のプロット

In [18]:
bound_list = [i / 100 for i in range(2, 101)]
lates = []
Ns = []
ses = []
for bound in bound_list:
    bounded_data = rdd_data[(rdd_data.history_log >= threshold_value - bound) & (rdd_data.history_log < threshold_value + bound)]
    agg_data = bounded_data.groupby('treatment').agg(count=('visit', 'count'), visit_rate=('visit', 'mean'))
    lates.append(agg_data.loc[1, 'visit_rate'] - agg_data.loc[0, 'visit_rate'])
    N = sum(agg_data['count'])
    Ns.append(N)
    ses.append(np.sqrt(sum(agg_data.visit_rate ** 2)) / np.sqrt(N))
    
result_data = pd.DataFrame({
    'bound': bound_list,
    'late': lates,
    'N': Ns,
    'se': ses
})

In [19]:
fig = px.line(result_data, x='bound', y='late', title='5.5 利用するデータの範囲と推定結果')
fig.add_trace(go.Scatter(
    x=result_data.bound,
    y=result_data.late - (1.96 * result_data.se),
    fill=None,
    mode='lines',
    line_color='indigo',
    ))
fig.add_trace(go.Scatter(
    x=result_data.bound,
    y=result_data.late + (1.96 * result_data.se),
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='indigo'))